In [1]:
# Build the Neural Network ，导入构建神经网络必要的库
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Get Device for Training 获取训练设备 CPU or GPU
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
# Define the Class  定义神经网络类
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()#将输入的图像展平为一维向量
        # 定义一个包含三个全连接层的神经网络
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),#第一个，将输入的784个像素点转换为512个像素点
            nn.ReLU(),
            nn.Linear(512, 512),#第二个，512个像素点不变
            nn.ReLU(),
            nn.Linear(512, 10),#第三个，将512个像素点转换为10个像素点，对应数据集里的10个类别
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)#利用归一化指数函数将输出转换为概率
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


In [6]:
# 以 3 幅大小为 28x28 的图像为样本
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# 对 nn.Flatten 层进行初始化，将每幅 28x28 的二维图像转换成一个包含 784 个像素值的连续数组（保持最小批次维度（dim=0））
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
# 线性层是一个利用其存储的权重和偏置对输入进行线性变换的模块
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
#使用ReLU激活函数
print(f"Before ReLU: {hidden1}\n\n")#打印激活前的张量值
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")#ReLU激活函数将所有负值转换为零

Before ReLU: tensor([[-0.1731, -0.5297,  0.4760, -0.1307, -0.6925,  0.1529,  0.2574,  0.4343,
          0.4245, -0.1047, -0.1392,  0.2974, -0.4514,  0.5146, -0.5924,  0.2685,
         -0.3137, -0.0582,  0.6242,  0.3661],
        [-0.6046,  0.2460,  0.2023,  0.2056, -0.1582, -0.1251, -0.1164,  0.5647,
          0.6052, -0.3545, -0.0267,  0.2248, -0.6838,  0.3021, -0.6478,  0.2053,
         -0.3345, -0.1805,  0.0174,  0.1722],
        [-0.2132, -0.2045,  0.2342, -0.1881, -0.4129, -0.4041,  0.0410,  0.4493,
          0.5612, -0.0177, -0.0012, -0.0074, -0.4159,  0.3742, -0.6163,  0.2103,
         -0.2913, -0.3911,  0.2648,  0.2478]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4760, 0.0000, 0.0000, 0.1529, 0.2574, 0.4343, 0.4245,
         0.0000, 0.0000, 0.2974, 0.0000, 0.5146, 0.0000, 0.2685, 0.0000, 0.0000,
         0.6242, 0.3661],
        [0.0000, 0.2460, 0.2023, 0.2056, 0.0000, 0.0000, 0.0000, 0.5647, 0.6052,
         0.0000, 0.0000, 0.2248, 0.0000, 0.3021, 0.00

In [10]:
# nn.Sequential 是一个有序的模块，数据按照定义的相同顺序通过模块
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [11]:
# 神经网络的最后一个线性层返回的是logits，这些值随后被传递给nn.Softmax，logits被缩放到[0, 1]范围内的值，表示模型对每个类别的预测概率。dim参数指定了值必须相加为1的维度。
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
# nn.Module 会自动跟踪模型对象内定义的所有字段，并使用模型的 parameters() 或 named_parameters() 方法访问所有参数
print(f"Model structure: {model}\n\n")
#遍历模型的所有参数
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0176,  0.0335,  0.0140,  ..., -0.0224, -0.0296,  0.0105],
        [ 0.0240, -0.0333,  0.0353,  ...,  0.0161,  0.0101,  0.0275]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0099,  0.0019], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0216,  0.0281,  0.0399,  ...,  0.0240,  0.0315,  0.0204],
        [ 0.0190, -0.0353, -0.0260,  ..., -0.0123, -0.0288, -0.0208]],
       device='cuda:0', grad_fn=<Sl